In [1]:
import sys
import os
import sqlite3 as sq
import pandas as pd

In [4]:
InputFileName1='Retrieve_DE_Billboard_Locations.csv'
InputFileName2='Retrieve_Online_Visitor.csv'
InputFileDir = 'C:/VKHCG/02-Krennwallner/01-Retrieve/01-EDS/02-Python'
OutputFileDir = 'C:/VKHCG/02-Krennwallner/02-Assess/01-EDS/02-Python'
OutputFileName='Assess-DE-Billboard-Visitor.csv'
DataBaseDir = 'C:/VKHCG/02-Krennwallner/02-Assess/SQLite'
DataBaseName='krennwallner.db'

In [7]:
if not os.path.exists(DataBaseDir):    
    os.makedirs(DataBaseDir) 

sDatabaseName=DataBaseDir + '/' + DataBaseName
conn = sq.connect(sDatabaseName)

In [8]:
conn = sq.connect(sDatabaseName)

In [9]:
### Import Billboard Data
sFileName= InputFileDir + '/' + InputFileName1
print('Loading :',sFileName)

Loading : C:/VKHCG/02-Krennwallner/01-Retrieve/01-EDS/02-Python/Retrieve_DE_Billboard_Locations.csv


In [10]:
BillboardRawData=pd.read_csv(sFileName,header=0,low_memory=False, encoding="latin-1")
BillboardRawData.head()

,Country,Place_Name,Latitude,Longitude
0,DE,Lake,51.7833,8.5667
1,DE,Horb,48.4333,8.6833
2,DE,Hardenberg,51.1000,7.7333
3,DE,Horn-bad Meinberg,51.9833,8.9667
4,DE,Winkel,51.5500,13.3833


In [11]:
BillboardRawData.shape

(8873, 4)

In [12]:
BillboardRawData.drop_duplicates(subset=None, keep='first', inplace=True)
BillboardData=BillboardRawData
BillboardData.shape

(8873, 4)

In [13]:
print('Loaded Company :',BillboardData.columns.values)

Loaded Company : ['Country' 'Place_Name' 'Latitude' 'Longitude']


In [15]:
sTable='Assess_BillboardData'
print('Storing :',sDatabaseName,' Table:',sTable)
BillboardData.to_sql(sTable, conn, if_exists="replace")

Storing : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  Table: Assess_BillboardData


In [21]:
print(BillboardData.head())

  Country         Place_Name  Latitude  Longitude
0      DE               Lake   51.7833     8.5667
1      DE               Horb   48.4333     8.6833
2      DE         Hardenberg   51.1000     7.7333
3      DE  Horn-bad Meinberg   51.9833     8.9667
4      DE             Winkel   51.5500    13.3833


In [17]:
### Import Billboard Data
sFileName= InputFileDir + '/' + InputFileName2
print('Loading :',sFileName)

Loading : C:/VKHCG/02-Krennwallner/01-Retrieve/01-EDS/02-Python/Retrieve_Online_Visitor.csv


In [18]:
VisitorRawData=pd.read_csv(sFileName,header=0,low_memory=False, encoding="latin-1")
VisitorRawData.drop_duplicates(subset=None, keep='first', inplace=True)
VisitorData=VisitorRawData[VisitorRawData.Country=='DE']
print('Loaded Company :',VisitorData.columns.values)

Loaded Company : ['Country' 'Place_Name' 'Latitude' 'Longitude' 'First_IP_Number'
 'Last_IP_Number']


In [19]:
sTable='Assess_VisitorData'
print('Storing :',sDatabaseName,' Table:',sTable)
VisitorData.to_sql(sTable, conn, if_exists="replace")

Storing : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  Table: Assess_VisitorData


In [20]:
print(VisitorData.head())

       Country Place_Name  Latitude  Longitude  First_IP_Number  \
368678      DE       Lake   51.7833     8.5667       1418011904   
368679      DE       Lake   51.7833     8.5667       1528171008   
368680      DE       Lake   51.7833     8.5667       1528177920   
408664      DE       Horb   48.4333     8.6833        777246720   
408665      DE       Horb   48.4333     8.6833       1339562496   

        Last_IP_Number  
368678      1418012159  
368679      1528171263  
368680      1528178175  
408664       777246975  
408665      1339562751  


In [22]:
VisitorData.shape

(75999, 6)

In [23]:
sTable='Assess_BillboardVisitorData'
print('Loading :',sDatabaseName,' Table:',sTable)

Loading : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  Table: Assess_BillboardVisitorData


In [24]:
sSQL="select distinct"
sSQL=sSQL+ " A.Country AS BillboardCountry,"
sSQL=sSQL+ " A.Place_Name AS BillboardPlaceName,"
sSQL=sSQL+ " A.Latitude AS BillboardLatitude, "
sSQL=sSQL+ " A.Longitude AS BillboardLongitude,"
sSQL=sSQL+ " B.Country AS VisitorCountry,"
sSQL=sSQL+ " B.Place_Name AS VisitorPlaceName,"
sSQL=sSQL+ " B.Latitude AS VisitorLatitude, "
sSQL=sSQL+ " B.Longitude AS VisitorLongitude,"
sSQL=sSQL+ " (B.Last_IP_Number - B.First_IP_Number) * 365.25 * 24 * 12 AS VisitorYearRate"
sSQL=sSQL+ " from"
sSQL=sSQL+ " Assess_BillboardData as A"
sSQL=sSQL+ " JOIN "
sSQL=sSQL+ " Assess_VisitorData as B"
sSQL=sSQL+ " ON "
sSQL=sSQL+ " A.Country = B.Country"
sSQL=sSQL+ " AND "
sSQL=sSQL+ " A.Place_Name = B.Place_Name;"
BillboardVistorsData=pd.read_sql_query(sSQL, conn)
print(BillboardVistorsData.head())

  BillboardCountry BillboardPlaceName  BillboardLatitude  BillboardLongitude  \
0               DE               Lake            51.7833              8.5667   
1               DE               Horb            48.4333              8.6833   
2               DE               Horb            48.4333              8.6833   
3               DE               Horb            48.4333              8.6833   
4               DE               Horb            48.4333              8.6833   

  VisitorCountry VisitorPlaceName  VisitorLatitude  VisitorLongitude  \
0             DE             Lake          51.7833            8.5667   
1             DE             Horb          48.4333            8.6833   
2             DE             Horb          48.4333            8.6833   
3             DE             Horb          48.4333            8.6833   
4             DE             Horb          48.4333            8.6833   

   VisitorYearRate  
0       26823960.0  
1       26823960.0  
2       53753112.0  
3 

In [25]:
BillboardVistorsData.shape

(181235, 9)

In [26]:
sTable='Assess_BillboardVistorsData'
print('Storing :',sDatabaseName,' Table:',sTable)
BillboardVistorsData.to_sql(sTable, conn, if_exists="replace")

Storing : C:/VKHCG/02-Krennwallner/02-Assess/SQLite/krennwallner.db  Table: Assess_BillboardVistorsData


In [27]:
sFileName=OutputFileDir + '/' + OutputFileName
print('Storing :', sFileName)

Storing : C:/VKHCG/02-Krennwallner/02-Assess/01-EDS/02-Python/Assess-DE-Billboard-Visitor.csv


In [28]:
BillboardVistorsData.to_csv(sFileName, index = False)
print('### Done!! #####################')

### Done!! #####################
